In [ ]:
!pip install bing-image-downloader
!pip install gradio


In [ ]:
from bing_image_downloader import downloader

foods = ["banh cuon Viet Nam", "pho Viet Nam", "com tam Viet Nam", "banh mi Viet Nam", "banh xeo Viet Nam"]

for food in foods:
    downloader.download(food, limit=100, output_dir='dataset', adult_filter_off=True, force_replace=False, timeout=60)

print("✅ Đã tải xong dữ liệu từ Bing")


[%] Downloading Images to /content/dataset/banh cuon Viet Nam


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://vietnamnomad.com/wp-content/uploads/2022/11/Banh-Cuon-A-guide-to-Vietnamese-steamed-rice-rolls-Vietnamnomad.jpg
[%] File Downloaded !

[%] Downloading Image #2 from http://static1.squarespace.com/static/5fee116050339566e9b38f09/5fee11df5abd827cad1a68c9/6202b310f8ff44002b245cc0/1644451163752/Vietnamese-Banh-Cuon-steamed+rice-rolls+-2.JPG?format=1500w
[Error]Invalid image, not saving http://static1.squarespace.com/static/5fee116050339566e9b38f09/5fee11df5abd827cad1a68c9/6202b310f8ff44002b245cc0/1644451163752/Vietnamese-Banh-Cuon-steamed+rice-rolls+-2.JPG?format=1500w

[!] Issue getting: http://static1.squarespace.com/static/5fee116050339566e9b38f09/5fee11df5abd827cad1a68c9/6202b310f8ff44002b245cc0/1644451163752/Vietnamese-Banh-Cuon-steamed+rice-rolls+-2.JPG?format=1500w
[!] Error:: Invalid image, not saving http://static1.squarespa

In [ ]:
import os, shutil
from sklearn.model_selection import train_test_split

base_dir = "dataset_split"
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    os.mkdir(os.path.join(base_dir, "train"))
    os.mkdir(os.path.join(base_dir, "test"))

    for food in foods:
        path = f"dataset/{food}"
        images = os.listdir(path)
        train_files, test_files = train_test_split(images, test_size=0.2, random_state=42)

        os.mkdir(os.path.join(base_dir, "train", food))
        os.mkdir(os.path.join(base_dir, "test", food))

        for f in train_files:
            shutil.copy(os.path.join(path, f), os.path.join(base_dir, "train", food))
        for f in test_files:
            shutil.copy(os.path.join(path, f), os.path.join(base_dir, "test", food))

print("✅ Đã chia train/test")


✅ Đã chia train/test


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    base_dir+"/train",
    target_size=(128,128),
    batch_size=16,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    base_dir+"/test",
    target_size=(128,128),
    batch_size=16,
    class_mode='categorical'
)


Found 391 images belonging to 5 classes.
Found 96 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

model_food = Sequential([
    Flatten(input_shape=(128,128,3)),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(len(foods), activation='softmax')
])

model_food.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_food.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 49152)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    25,166,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,298,949 (96.51 MB)

 Trainable params: 25,298,949 (96.51 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("food_ann_best.h5", monitor="val_accuracy", save_best_only=True, mode="max")
earlystop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model_food.fit(
    train_data,
    validation_data=test_data,
    epochs=20,
    callbacks=[checkpoint, earlystop]
)

print("✅ Train xong, model đã lưu thành công!")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 884ms/step - accuracy: 0.2289 - loss: 25.6515

25/25 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.2284 - loss: 25.5470 - val_accuracy: 0.2500 - val_loss: 8.0620
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2392 - loss: 10.2511

25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.2391 - loss: 10.1606 - val_accuracy: 0.2708 - val_loss: 2.2427
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.2358 - loss: 3.2088 - val_accuracy: 0.2708 - val_loss: 1.8355
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 984ms/step - accuracy: 0.2760 - loss: 1.8363 - val_accuracy: 0.2292 - val_loss: 1.5643
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 842ms/step - accuracy: 0.2973 - loss: 1.5494

25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.2970 - loss: 1.5497 - val_accuracy: 0.2917 - val_loss: 1.5273
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 896ms/step - accuracy: 0.3011 - loss: 1.5131

25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.3001 - loss: 1.5136 - val_accuracy: 0.3021 - val_loss: 1.5031
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 906ms/step - accuracy: 0.2969 - loss: 1.5099

25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.2968 - loss: 1.5103 - val_accuracy: 0.3333 - val_loss: 1.4935
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 972ms/step - accuracy: 0.2554 - loss: 1.5023 - val_accuracy: 0.3125 - val_loss: 1.4725
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 836ms/step - accuracy: 0.3073 - loss: 1.4659

25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.3064 - loss: 1.4662 - val_accuracy: 0.3438 - val_loss: 1.4777
Epoch 10/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 37s 973ms/step - accuracy: 0.3102 - loss: 1.4513 - val_accuracy: 0.3125 - val_loss: 1.4500
Epoch 11/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 892ms/step - accuracy: 0.3236 - loss: 1.4385

25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.3234 - loss: 1.4386 - val_accuracy: 0.3542 - val_loss: 1.4633
Epoch 12/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 982ms/step - accuracy: 0.3073 - loss: 1.4541 - val_accuracy: 0.3125 - val_loss: 1.5136
Epoch 13/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 956ms/step - accuracy: 0.2929 - loss: 1.4903 - val_accuracy: 0.2917 - val_loss: 1.5112
Epoch 14/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 912ms/step - accuracy: 0.2352 - loss: 1.5337 - val_accuracy: 0.3333 - val_loss: 1.4457
Epoch 15/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 980ms/step - accuracy: 0.3588 - loss: 1.3849 - val_accuracy: 0.3021 - val_loss: 1.4819
Epoch 16/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 846ms/step - accuracy: 0.3226 - loss: 1.4646

25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.3220 - loss: 1.4644 - val_accuracy: 0.3646 - val_loss: 1.4454
Epoch 17/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 971ms/step - accuracy: 0.2536 - loss: 1.5273 - val_accuracy: 0.3229 - val_loss: 1.4283
Epoch 18/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 982ms/step - accuracy: 0.3068 - loss: 1.4067 - val_accuracy: 0.3125 - val_loss: 1.4899
Epoch 19/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 956ms/step - accuracy: 0.2612 - loss: 1.4659 - val_accuracy: 0.3333 - val_loss: 1.4895
Epoch 20/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 913ms/step - accuracy: 0.3177 - loss: 1.4315 - val_accuracy: 0.3333 - val_loss: 1.4532
✅ Train xong, model đã lưu thành công!


In [ ]:
import pickle

# Lưu labels
with open("/content/drive/MyDrive/food_labels.pkl", "wb") as f:
    pickle.dump(list(train_gen.class_indices.keys()), f)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/food_labels.pkl'

In [ ]:
# Lưu model đã train vào Drive
model_food.save("/content/drive/MyDrive/food_ann_best.h5")
print("✅ Model đã được lưu vào Google Drive!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.models import load_model
model_food = load_model("/content/drive/MyDrive/food_ann_best.h5")

print("✅ Đã load lại mô hình thành công!")


In [ ]:
from tensorflow.keras.models import load_model
model_food = load_model("food_ann_best.h5")
print("✅ Đã load lại model")


In [ ]:
from tensorflow.keras.models import load_model
import pickle

# Load model
model = load_model("/content/drive/MyDrive/food_ann_best.h5")

# Load labels
with open("/content/drive/MyDrive/food_labels.pkl", "rb") as f:
    labels = pickle.load(f)

print("✅ Model và labels đã load:", labels)


In [ ]:
import gradio as gr
import numpy as np

img_size = (128,128)

def predict_food(img):
    img = img.resize(img_size)
    x = np.array(img)/255.0
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    return {labels[i]: float(preds[0][i]) for i in range(len(labels))}

iface = gr.Interface(
    fn=predict_food,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="🍜 Nhận diện món ăn Việt Nam",
    description="Tải ảnh món ăn (Bánh cuốn, Phở, Cơm tấm, Bánh mì, Bánh xèo) để nhận diện."
)

iface.launch(debug=True)
